# Project 7

Grid Search + Parameter Tuning + DropOut in Iris dataset

In [3]:
!pip install skorch

In [25]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch import optim
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from skorch import NeuralNetClassifier



In [5]:
df = pd.read_csv('/content/iris.csv')

In [6]:
np.random.seed(123)
torch.manual_seed(123)

In [7]:
forecasters = df.iloc[:, 0:4].values
labels = df.iloc[:, 4].values

In [8]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

In [9]:
forecasters = np.array(forecasters, dtype='float32')
labels = labels.astype('int64')

In [10]:
class torch_classifier(nn.Module):
  def __init__(self, activation, neurons, initializer, dropout):
    super().__init__()

    self.dense0 = nn.Linear(4, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation

    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 3)

    self.dropout = nn.Dropout(dropout)

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dropout(X)

    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dropout(X)

    X = self.dense2(X)

    return X


In [11]:
sklearn_classifier = NeuralNetClassifier(module=torch_classifier,
                                         criterion = torch.nn.CrossEntropyLoss,
                                         train_split=False)

In [15]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam],
          'module__activation': [F.relu],
          'module__neurons': [4, 8],
          'module__initializer': [torch.nn.init.uniform_],
          'module__dropout': [0.2, 0.3]
          }

In [16]:
grid_search = GridSearchCV(estimator = sklearn_classifier,
                           param_grid = params,
                           scoring = 'accuracy',
                           cv = 3)

In [17]:
grid_search = grid_search.fit(forecasters, labels);

  epoch    train_loss     dur
-------  ------------  ------
      1       10.5902  0.0174
      2        5.1621  0.0185
      3        2.8981  0.0166
      4        2.1835  0.0183
      5        1.7394  0.0176
      6        1.8199  0.0164
      7        1.5159  0.0184
      8        1.2023  0.0167
      9        1.4016  0.0174
     10        1.2750  0.0166
     11        1.0696  0.0242
     12        1.1939  0.0209
     13        1.1130  0.0162
     14        1.0233  0.0174
     15        1.0344  0.0192
     16        0.9520  0.0171
     17        0.9782  0.0178
     18        0.9875  0.0200
     19        0.9993  0.0168
     20        0.9127  0.0165
     21        0.9126  0.0231
     22        0.9506  0.0170
     23        0.9395  0.0160
     24        0.8545  0.0172
     25        0.8596  0.0170
     26        0.8302  0.0173
     27        0.8161  0.0172
     28        0.8173  0.0167
     29        0.7735  0.0164
     30        0.8056  0.0190
     31        0.7376  0.0176
     32   

In [18]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [20]:
best_parameters, best_accuracy

({'batch_size': 10,
  'max_epochs': 100,
  'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'module__dropout': 0.2,
  'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
  'module__neurons': 8,
  'optimizer': torch.optim.adam.Adam},
 0.9933333333333333)

---

In [21]:
forecasters = torch.tensor(forecasters, dtype= torch.float)
labels = torch.tensor(labels, dtype=torch.long)

In [22]:
dataset = torch.utils.data.TensorDataset(
    forecasters,
    labels
)

train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=10,
    shuffle=True
)

In [23]:
classifier = nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(8,8),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(8,3)
)

In [27]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(classifier.parameters(),
                       lr = 0.001,
                       weight_decay=0.0001)

In [29]:
for epoch in range(100):

  running_loss = 0.
  running_accuracy = 0.

  for data in train_loader:
    inputs, label = data

    optimizer.zero_grad()

    outputs = classifier(inputs)

    loss = criterion(outputs, label)
    loss.backward()

    optimizer.step()

    running_loss += loss.item()

    ps = F.softmax(outputs)
    top_p, top_class = ps.topk(k=1, dim=1)

    equals = top_class == label.view(*top_class.shape)
    running_accuracy += torch.mean(equals.type(torch.float))

  print('Epoch {:3d}: Loss {:3.5f} - Accuracy {:3.5}'.format(epoch +1, running_loss/len(train_loader), running_accuracy/len(train_loader)))

<ipython-input-29-67169367f3e1>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ps = F.softmax(outputs)


Epoch   1: Loss 1.18153 - Accuracy 0.34667
Epoch   2: Loss 1.14418 - Accuracy 0.36
Epoch   3: Loss 1.10241 - Accuracy 0.38
Epoch   4: Loss 1.11799 - Accuracy 0.35333
Epoch   5: Loss 1.11237 - Accuracy 0.36667
Epoch   6: Loss 1.08526 - Accuracy 0.4
Epoch   7: Loss 1.06622 - Accuracy 0.38667
Epoch   8: Loss 1.09126 - Accuracy 0.38
Epoch   9: Loss 1.09909 - Accuracy 0.34
Epoch  10: Loss 1.09519 - Accuracy 0.34
Epoch  11: Loss 1.02333 - Accuracy 0.45333
Epoch  12: Loss 1.04945 - Accuracy 0.39333
Epoch  13: Loss 1.02735 - Accuracy 0.38667
Epoch  14: Loss 1.01069 - Accuracy 0.45333
Epoch  15: Loss 1.02261 - Accuracy 0.41333
Epoch  16: Loss 0.98194 - Accuracy 0.47333
Epoch  17: Loss 0.99294 - Accuracy 0.47333
Epoch  18: Loss 0.96169 - Accuracy 0.52667
Epoch  19: Loss 0.92486 - Accuracy 0.52
Epoch  20: Loss 0.98922 - Accuracy 0.48667
Epoch  21: Loss 0.90212 - Accuracy 0.57333
Epoch  22: Loss 0.91698 - Accuracy 0.58667
Epoch  23: Loss 0.92512 - Accuracy 0.56667
Epoch  24: Loss 0.89319 - Accurac

In [30]:
torch.save(classifier.state_dict(), "iris.pth")

In [31]:
state_dict = torch.load('/content/iris.pth')
classifier.load_state_dict(state_dict)

<All keys matched successfully>

In [32]:
new = torch.tensor([[3.2, 4.5, 0.9, 1.1]], requires_grad=False)

In [33]:
classifier.eval()

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=8, out_features=8, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=8, out_features=3, bias=True)
)

In [35]:
forecasting = classifier(new)
forecasting = F.softmax(forecasting)
forecasting = (forecasting > 0.5).numpy()

<ipython-input-35-7e0636ab488f>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  forecasting = F.softmax(forecasting)


In [36]:
if forecasting[0][0] == True and forecasting[0][1] == False and forecasting[0][2] == False:
    print('Iris setosa')
elif forecasting[0][0] == False and forecasting[0][1] == True and forecasting[0][2] == False:
    print('Iris virginica')
elif forecasting[0][0] == False and forecasting[0][1] == False and forecasting[0][2] == True:
    print('Iris versicolor')

Iris setosa
